In [1]:
import xml.etree.ElementTree as ET
import os
from tqdm.notebook import tqdm
from PIL import Image
import sys
import numpy as np
import csv
%matplotlib inline

In [2]:
trainPath = 'Train'
trainLabelPath = 'Train/labelXML'
trainImagePath = 'Train/images'

In [3]:
class Point:
    def __init__(self, x,y):
        self.x = x
        self.y = y

class Points:
    def __init__(self):
        self.points = []
        self.min_x = sys.maxsize
        self.min_y = sys.maxsize
        self.max_x = 0
        self.max_y = 0
    def __len__(self):
        return len(self.points)
    def add_point(self, x,y):
        self.min_x = min(x,self.min_x)
        self.min_y = min(y,self.min_y)
        self.max_x = max(x,self.max_x)
        self.max_y = max(y,self.max_y)
        self.points.append(Point(x,y))
    

class Item:
    def __init__(self, filename, label, points):
        self.filename = filename
        self.label = label
        self.points = points

## List of aircraft in set
* Boeing787
* Boeing747
* C919
* A321
* A220
* Boeing737
* A350
* A330
* Boeing777
* ARJ21
* other-airplane

In [4]:
directory = os.fsencode(trainLabelPath)

known_aircraft = {"Boeing787", "Boeing747", "C919", "A321", "A220", "Boeing737", "A350", "A330", 
                  "Boeing777", "ARJ21", "other-airplane"}
countsDict = {}
to_delete = set([])
total = 0
for item in known_aircraft:
    countsDict[item] = 0

labels = {}
labelSelection = set()

for file in tqdm(os.listdir(directory)):
    filename = os.fsdecode(file)
    if filename.endswith(".xml"):
        tree = ET.parse(f'{trainLabelPath}/{filename}')
        root = tree.getroot()
        filename = root[0][0].text
        
        objects = {}
        i = 0
        for obj in root[3]:
            label = obj[3][0].text
            if not label in known_aircraft:
                continue
            total += 1
            countsDict[label] += 1
            labelSelection.add(label)
            points = Points()
            for point in obj[4]:
                split=point.text.split(',')
                points.add_point(float(split[0]),float(split[1]))
            new_itm = Item(filename, label, points)
            objects[i] = new_itm
            i += 1
    else:
        continue
    if len(objects) > 0:
        labels[filename.replace('.tif',"")] = objects
    else:
        to_delete.add(filename.replace('.tif',""))
print(len(labels))
print(total)
print(len(to_delete))
print(labelSelection)
print(countsDict)

  0%|          | 0/5592 [00:00<?, ?it/s]

5591
30324
0
{'Boeing777', 'ARJ21', 'Boeing747', 'A321', 'Boeing787', 'C919', 'other-airplane', 'Boeing737', 'A350', 'A220', 'A330'}
{'Boeing777': 1532, 'ARJ21': 166, 'Boeing747': 1673, 'Boeing787': 1669, 'A321': 2505, 'C919': 135, 'other-airplane': 9975, 'Boeing737': 3949, 'A350': 1064, 'A220': 6057, 'A330': 1599}


In [5]:
for file in tqdm(to_delete):
    pathImg = f'{trainImagePath}/{file}.tif'
    pathXml = f'{trainLabelPath}/{file}.xml'
    if os.path.exists(pathImg):
        print(f'Removing {pathImg}')
        os.remove(pathImg)
    if os.path.exists(pathXml):
        print(f'Removing {pathXml}')
        os.remove(pathXml)

0it [00:00, ?it/s]

In [11]:
new_path = 'Train/in_images'
grey_path = 'Train/in_grey_images'
size = 128,128
keys = list(labels.keys())
test = keys[0]

In [21]:
data= []
header = ['file', 'label']
rgb_weights = [0.2989, 0.5870, 0.1140]

for key in tqdm(keys):
    filename=f'{key}.tif'
    path = f'{trainImagePath}/{filename}'
    im = Image.open(path)
    
    for obj in labels[key]:
        points = labels[key][obj].points
        im1 = im.crop((points.min_x, points.min_y, points.max_x, points.max_y)).resize(size)
        newfilename = f'{key}_{obj}.png'
        data.append([newfilename, label])
        im1.save(f'{new_path}/{newfilename}')
        im1=np.array(im1)
        im1 = np.dot(im1[...,:3], rgb_weights)
        im1 = Image.fromarray(im1).convert('RGB')
        im1.save(f'{grey_path}/{newfilename}')

with open('Train/train.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    for row in data:
        writer.writerow(row)

  0%|          | 0/5591 [00:00<?, ?it/s]

In [25]:
header = ['file', 'label']
data= []
for key in tqdm(keys):
    for obj in labels[key]:
        filename = f'{key}_{obj}.png'
        label = labels[key][obj].label
        data.append([filename, label])

with open('Train/train.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    for row in data:
        writer.writerow(row)

  0%|          | 0/5591 [00:00<?, ?it/s]